# Loading and Preprocessing Data with TensorFlow

In [29]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import pandas as pd

## The Data API

The whole Data api revolves around the concept of *datasets*. Usually, we will use datasets that gradually read data from disk, but from simplicity let's create a dataset entirely in RAM using `tf.data.Dataset.from_tensor_slices()`:

In [14]:
X = tf.range(10) # any data tensor
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

The `from_tensor_slices()` function takes a tensor and creates a `tf.data.Dataset` whose elements are all slices of `X` (along the first dimension), so this dataset contains 10 items: tensors 0, 1, 2,...,9. In this case we would have obtained the same dataset if we had used `tf.data.Dataset.range(10)`

We can simply iterate over a dataset's items like this:

In [15]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


### Chaining Transformations

Once we have dataset, we can apply all sort of transformations to it by calling its transformation methods. Each method returns a new dataset, so we can chain transformations like this:

In [16]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In this example, we first call `repeat()` method on the original dataset, and it returns a new dataset that will repeat the items in the original dataset three times. Then we call the `batch()` method on this new dataset, and again creates a new dataset. This one will group the items of the previous dataset in batches of seven items. Finally, we can iterate over the items of this final dataset.

As we can see, the `batch()` method had to output a final batch of size two instead of seven, but we can call it with a `drop_remainder=True` if we want it to drop this final batch so that all batches have the exact same size. 

We can also transform the items by calling the `map()` method. For ex., this creates a new dataset with all items doubled:

In [17]:
dataset = dataset.map(lambda x: x*2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


This function is the one we will call to apply any preprocessing we want to our data. Sometimes this will include computations that can be quite intensive, such as reshaping or rotating an image, so we will usually want to spawn multiple threads to speed things up: it's as simple as setting the
`num_parallel_calls` argument. 

**NOTE**:
The function we pass to the `map()` must be convertible to TF Function

While `map()` method applies transformation to each item, the `apply()` method applies a transformation to the dataset as a whole. 

For ex., the `unbatch()` function to the dataset will create a new dataset where each item will be single-integer tensor instead of batch of seven integers:

In [18]:
dataset = dataset.unbatch()
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, sh

It is also possible to simply filter the dataset using the `filter()` method:

In [19]:
dataset = dataset.filter(lambda x: x < 10)
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


We will often want to look at just few items from a dataset. We can use `take()` method for that:

In [20]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


### Shuffling the Data

Gradient Descent works best when the instances in the training set are independent and identically distributed. A simple way to ensure this is to shuffle the instances using the `shuffle()` method.  For ex:

In [21]:
dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, 3 times
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7) # we must specify the buffer size, and it is imp to make it large enough, or else shuffling will not be very effective. 
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


**TIP:**

If we call `repeat()` on a shuffled dataset, by default it will generate a new order at every iteration.

For large datasets that does not fit in memory, this simple shuffling-buffer approach may not be sufficient, since the buffer will be small compared to dataset. One solution is to shuffle the source data itself. This will definitely improve the shuffling a lot. Even if the data is shuffled, we usually want to shuffle it more. To shuffle the instances some more, a common approach is to split source data into multiple files, then read them in random order during training. However, instances located in same file will still end up close together. To avoid this, we can pick multiple files randomly and read them simultaneously, interleaving records. Then on top of that we can add a shuffling buffer using `shuffle()` method. Let's see how it works:

Let's start by loading and preparing California dataset. We will first load it, split it intro training, validation and test set and finally we scale it.

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data,
                                                              housing.target.reshape(-1,1),
                                                             random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

For very large dataset that does not fit in memory, we will typically need to split it into many files first, then have TF read these files in parallel. To demonstrate this, let's start by splitting the housing dataset and save it in 20 csv files:

In [3]:
def save_to_multiple_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join(os.curdir, "datasets", "california_housing")
    # print(housing_dir)
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")
    
    file_paths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        file_paths.append(part_csv)
        with open (part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                """
                Python repr() function returns a printable representation of the object by converting
                that object to a string.
                """
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return file_paths

In [4]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_files(test_data, "test", header, n_parts=10)

In [5]:
train_filepaths

['./datasets/california_housing/my_train_00.csv',
 './datasets/california_housing/my_train_01.csv',
 './datasets/california_housing/my_train_02.csv',
 './datasets/california_housing/my_train_03.csv',
 './datasets/california_housing/my_train_04.csv',
 './datasets/california_housing/my_train_05.csv',
 './datasets/california_housing/my_train_06.csv',
 './datasets/california_housing/my_train_07.csv',
 './datasets/california_housing/my_train_08.csv',
 './datasets/california_housing/my_train_09.csv',
 './datasets/california_housing/my_train_10.csv',
 './datasets/california_housing/my_train_11.csv',
 './datasets/california_housing/my_train_12.csv',
 './datasets/california_housing/my_train_13.csv',
 './datasets/california_housing/my_train_14.csv',
 './datasets/california_housing/my_train_15.csv',
 './datasets/california_housing/my_train_16.csv',
 './datasets/california_housing/my_train_17.csv',
 './datasets/california_housing/my_train_18.csv',
 './datasets/california_housing/my_train_19.csv']

Okay, now let's look at the first few lines of one of these csv files:

In [48]:
pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


Or in text mode:

In [51]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


#### Interleaving lines from multiple files

Now let's create a dataset containing only these train file paths:

In [54]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

By defult `list_files()` returns a dataset that shuffles the file paths. In general this is good, but we can set `shuffle=False` if we do not want that for some reason.

Now we can call the `interleave()` method to read from five files at a time and interleave thier lines (skipping the first line of each file, which is header row, using the `skip()` method):

In [56]:
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'./datasets/california_housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'./datasets/california_housing/my_train_09.csv', shape=(), dtype=

In [55]:
n_readers = 5
"""
The tf.data.TextLineDataset loads text from text files and creates a dataset where each line of the files becomes an element of the dataset.
"""
dataset = filepath_dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
                                  cycle_length=n_readers)

The `interleave()` method will create a dataset that will pull five file path from the `filepath_dataset`, and for each one it will call the function we gave it (a lambda in this example), to create a new dataset (in this case a `TextLineDataset`). 

To be clear, at this stage there will be seven datasets in all: the filepath dataset, the interleave dataset, and the five `TextLineDataset` created internally by the interleave dataset. When we interate over the interleave dataset, it will cycle through these five `TextLineDatasets`, reading one line at a time from each until all datasets are out of items. Then it will get next five file paths from the `filepath_datasets` and interleave them the same way, and so on until it runs out of file paths. 

**TIP:**

For interleaving to work best, it is preferable to have files of identical lengths; otherwise the ends of the longest files will not be interleaved.

By default, `interleave()` does not use parallelism; it just reads one line at a time from each line, sequentially. If we want it to actually read files in parallel, we can set the `num_parallel_calls` argument to the number of threads we want. We can even set  it to `tf.data.AUTOTUNE` to make tf choose the right number of threads dynamically based on the available CPU. 

Let's look at what the dataset contains now:

In [60]:
for line in dataset.take(5):
    print(line.numpy())

b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215'
b'3.226,52.0,5.372469635627531,0.9473684210526315,1157.0,2.3421052631578947,37.96,-121.31,1.076'
b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67'
b'3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442'
b'3.0217,22.0,4.983870967741935,1.1008064516129032,615.0,2.4798387096774195,38.76,-120.6,1.069'


Looks good! But as we can see, these are just byte strings; we need to parse them and scale the data.

In [63]:
record_defaults = [0, np.nan, tf.constant(np.nan, dtype=np.float64), "Hello", tf.constant([])]
"""
tf.io.decode_csv : Convert CSV records to tensors. Each column maps to one tensor.

record_defaults = A list of Tensor objects with specific types. 
"""
parsed_fields = tf.io.decode_csv("1,2,3,4,5", record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
 <tf.Tensor: shape=(), dtype=float64, numpy=3.0>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'4'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

Notice that in above, field 4 is interpreted as string.

In [64]:
parsed_fields = tf.io.decode_csv(",,,,5", record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float64, numpy=nan>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'Hello'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

Notice that all the missing values are replaced by default values.

Also the number of fields should match exactly the number of fields in record_defaults

### Preprocessing the Data

Now let's implement a function that perfroms this preprocessing (parsing the csv and scaling them):

In [9]:
n_inputs = X_train.shape[-1]

def preprocess(line):
    defaults = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defaults)
    # print(fields)
    # fields is a list of tensors where each tensor contains one feature value. So we need to use tf.stack to make it to array
    x = tf.stack(fields[:-1])
    # print(x)
    y = tf.stack(fields[-1:])
    return (x-X_mean) / X_std, y

In [89]:
preprocess(b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73, -118.31,3.215')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.15159765,  1.8491895 ,  0.09624147, -0.22151624, -0.66828614,
        -0.23549314, -0.89780813,  0.6368871 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.215], dtype=float32)>)

Looks good! We can now apply the function to the dataset.

### Putting EveryThing Together

To make the code reusable, let's put together everything in small helper function: it will create and return a dataset that will efficiently load California housing dataset from multiple CSV files, preprocess it, shuffle it and optionally repeat it, and batch it.

In [7]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5, n_read_threads=None,
                       shuffle_buffer_size=10000, n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
                                cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

The last line `prefetch(1)` is important for performance. 

### Prefetching

By calling `prefetch(1)` at the end, we are creating a dataset that will do its best to always be one batch ahead. (In general prefetching one batch is fine, but in some case we may need few more. We can always let tf decide on its own by passign `tf.data.AUTOTUNE`). In other words, while our training algorithm is working on one batch, the dataset will already be working in parallel on getting the next batch ready (e.g., reading the data from disk and preprocessing it). This can improve performace dramatically. If we also ensure that loading and preprocessing are multithreaded, we can exploit multiple cores on the CPU. 

If dataset is small enough to fit in memory, we can significantly speed up training by using the dataset's `cache()` method to cache its content to RAM. We should generally do this after loading and preprocessing data, but before shuffling, repeating, batching and prefetching. This way, each instance will only be read and preprocessed once (instead once per epoch), but the data will still be shuffled differently at each epoch, and the next batch will still be prepared in advance. 

### Using the Dataset with tf.keras

Now we have `csv_reader_dataset()` function to create a dataset for the training set.

In [10]:
tf.random.set_seed(42)

train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch, y_batch in train_set.take(3):
    print("X =", X_batch)
    print("y =", y_batch)
    print()

X = tf.Tensor(
[[ 0.5804519  -0.20762321  0.05616303 -0.15191229  0.01343246  0.00604472
   1.2525111  -1.3671792 ]
 [ 5.818099    1.8491895   1.1784915   0.28173092 -1.2496178  -0.3571987
   0.7231292  -1.0023477 ]
 [-0.9253566   0.5834586  -0.7807257  -0.28213993 -0.36530012  0.27389365
  -0.76194876  0.72684526]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[1.752]
 [1.313]
 [1.535]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[-0.8324941   0.6625668  -0.20741376 -0.18699841 -0.14536144  0.09635526
   0.9807942  -0.67250353]
 [-0.62183803  0.5834586  -0.19862501 -0.3500319  -1.1437552  -0.3363751
   1.107282   -0.8674123 ]
 [ 0.8683102   0.02970133  0.3427381  -0.29872298  0.7124906   0.28026953
  -0.72915536  0.86178064]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[0.919]
 [1.028]
 [2.182]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[-1.0344558   1.0581076  -0.8869343  -0.08743899  0.6157541   0.4368748
  -0.75726473  0.64688075]
 [ 0.4675818   0.6625668   0.03120424 -0.

In [11]:
train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

And now we can simply build and train a Keras model using these datasets (support is specific for `tf.keras`). All we need to do is pass the training and validation datasets to the `fit()` method, instead of `X_train, y_train, X_vaild` and `y_valid`

In [12]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

In [13]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [14]:
batch_size=32
model.fit(train_set, steps_per_epoch=len(X_train)//batch_size, epochs=10,
         validation_data=valid_set)

Epoch 1/10
362/362 [==============================] - 1s 3ms/step - loss: 2.0315 - val_loss: 3.7452
Epoch 2/10
362/362 [==============================] - 1s 3ms/step - loss: 0.9367 - val_loss: 0.7827
Epoch 3/10
362/362 [==============================] - 1s 3ms/step - loss: 0.7497 - val_loss: 0.8117
Epoch 4/10
362/362 [==============================] - 1s 4ms/step - loss: 0.6687 - val_loss: 0.6846
Epoch 5/10
362/362 [==============================] - 1s 2ms/step - loss: 0.6179 - val_loss: 0.6467
Epoch 6/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6187 - val_loss: 0.6151
Epoch 7/10
362/362 [==============================] - 1s 4ms/step - loss: 0.5609 - val_loss: 0.7972
Epoch 8/10
362/362 [==============================] - 1s 2ms/step - loss: 0.5372 - val_loss: 0.5258
Epoch 9/10
362/362 [==============================] - 1s 4ms/step - loss: 0.5180 - val_loss: 0.5510
Epoch 10/10
362/362 [==============================] - 1s 4ms/step - loss: 0.5200 - val_loss: 0.4858

In [15]:
model.evaluate(test_set, steps=len(X_test) // batch_size)

161/161 [==============================] - 0s 1ms/step - loss: 0.4921


0.49213576316833496

In [19]:
new_set = test_set.take(3).map(lambda X, y: X) # pretend we have 3 new instances
model.predict(new_set)

3/3 [==============================] - 0s 8ms/step


array([[1.7322209 ],
       [1.3085666 ],
       [1.5255079 ],
       [1.0248382 ],
       [2.5808408 ],
       [2.2871757 ],
       [1.7916274 ],
       [4.3145485 ],
       [2.206674  ],
       [2.4540796 ],
       [2.491219  ],
       [2.857857  ],
       [2.7704601 ],
       [2.5426204 ],
       [0.67148834],
       [2.1122987 ],
       [3.4075441 ],
       [2.0837922 ],
       [5.348416  ],
       [1.9640238 ],
       [3.9412053 ],
       [1.339951  ],
       [2.236424  ],
       [1.6961956 ],
       [2.6966777 ],
       [1.5503691 ],
       [1.9002333 ],
       [2.3937144 ],
       [3.0539567 ],
       [1.2766786 ],
       [2.2559    ],
       [1.3738415 ],
       [2.22063   ],
       [0.91933674],
       [5.45888   ],
       [2.0175803 ],
       [1.8049254 ],
       [2.058518  ],
       [1.6453042 ],
       [3.0500243 ],
       [1.1761464 ],
       [1.2705905 ],
       [4.044486  ],
       [2.1326566 ],
       [1.2013569 ],
       [3.055455  ],
       [2.1411066 ],
       [1.086

If we want to build our own custom training loop, we can just iterate over the training set, very naturally:

In [22]:
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error

n_epochs = 5
batch_size = 32
n_steps_per_epoch = len(X_train) // batch_size
total_steps = n_epochs * n_steps_per_epoch
global_step = 0

for X_batch, y_batch in train_set.take(total_steps):
    global_step += 1
    print(f"\rGlobal step {global_step}/{total_steps}", end="")
    with tf.GradientTape() as tape:
        y_pred = model(X_batch)
        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

Global step 1810/1810

In fact, it is even possible to create TF Function, that performs the whole training loop:

In [23]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error

@tf.function
def train(model, n_epochs, batch_size=32, n_readers=5, n_read_threads=5,
         shuffle_buffer_size = 10000, n_parse_threads=5):
    train_set = csv_reader_dataset(train_filepaths, repeat=n_epochs, n_readers=n_readers,
                                  n_read_threads=n_read_threads,
                                  shuffle_buffer_size=shuffle_buffer_size,
                                  n_parse_threads=n_parse_threads, batch_size=batch_size)
    for X_batch, y_batch in train_set:
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))


In [24]:
train(model, 5)

## The TFRecord Format

We can easily create a TFRecord file using the `tf.io.TFRecordWriter` class:

In [30]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is first record")
    f.write(b"And this is second record")

And we can then use a `tf.data.TFRecordDataset` to read one or more TFRecord files:

In [31]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is first record', shape=(), dtype=string)
tf.Tensor(b'And this is second record', shape=(), dtype=string)


**TIP:**

By default, a `TFRecordDataset` will read files one by one, but we can make it read multiple files in parallel and interleave their records by setting `num_parallel_reads`.